In [1]:
import cv2
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


base_dir = './dataset'
os.makedirs(base_dir, exist_ok=True)


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

while True:
    
    person = input("Enter the name of the person (or type 'done' to finish): ").strip()
    if person.lower() == 'done':
        print("Dataset collection complete!")
        break

    
    person_dir = os.path.join(base_dir, person)
    if os.path.exists(person_dir):
        print(f"Images for {person} already exist. Skipping...")
        continue
    os.makedirs(person_dir, exist_ok=True)

    print(f"Collecting images for {person}. Press 'q' to quit.")
    count = 0

    while count < 100:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame from webcam.")
            break

        # Convert to grayscale for Haar Cascade
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

        for (x, y, w, h) in faces:
            
            padding = 10
            x1, y1 = max(0, x - padding), max(0, y - padding)
            x2, y2 = min(frame.shape[1], x + w + padding), min(frame.shape[0], y + h + padding)
            face_roi = frame[y1:y2, x1:x2]

            # Save the cropped face image
            img_path = os.path.join(person_dir, f"{person}_{count}.jpg")
            cv2.imwrite(img_path, face_roi)
            count += 1
            print(f"Captured {count}/100 for {person}")

            if count >= 100:
                break

        # Display the video feed
        cv2.imshow("Capture - Press 'q' to Quit", frame)

        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print(f"Stopped capturing for {person}.")
            break

cap.release()
cv2.destroyAllWindows()


Error: Could not access the webcam.


: 

In [ ]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from torch import to_categorical


IMG_SIZE = 128  # Resize images to 128x128
DATA_DIR = './dataset'

# Automatically get folder names as labels
persons = [name for name in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, name))]
labels = {name: i for i, name in enumerate(persons)}  # Assign labels to persons

# Load and preprocess images
X, y = [], []
for person in persons:
    folder = os.path.join(DATA_DIR, person)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        try:
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                X.append(img)
                y.append(labels[person])
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

X = np.array(X) / 255.0  # Normalize pixel values to [0, 1]
y = np.array(y)

if len(X) == 0:
    print("No valid data found. Exiting...")
    exit()

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=len(persons))
y_val = to_categorical(y_val, num_classes=len(persons))

print("Dataset preprocessed successfully!")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(persons), activation='softmax')  # Output layer
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=16)

# Save the trained model
model.save('attendance_cnn_model.h5')
print("Model trained and saved successfully!")


In [ ]:
from tensorflow.keras.models import load_model

# Load trained model
model = load_model('attendance_cnn_model.h5')

# Start webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    img = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img = np.expand_dims(img, axis=0) / 255.0

    # Predict the person
    predictions = model.predict(img)
    person_index = np.argmax(predictions)
    confidence = np.max(predictions)

    if confidence > 0.7:
        label = list(labels.keys())[person_index]
    else:
        label = "Unidentified"

    # Display result
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()
